***Задача:*** проанализировать базу данных

***Исходные данные:*** информация о книгах, издательствах, авторах, а также пользовательские обзоры книг

***Цель:*** Сделать анализ базы данных, который помжет сформулировать ценностное предложение для нового продукта

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

***Таблица books***

In [3]:
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


***Таблица authors***

In [4]:
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


***Таблица publishers***

In [5]:
query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


***Таблица ratings***

In [6]:
query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


***Таблица reviews***

In [7]:
query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


***1. Посчитайте, сколько книг вышло после 1 января 2000 года;***


In [8]:
query = ''' SELECT COUNT(book_id)
            FROM BOOKS
            WHERE publication_date > '2000-01-01'
        '''
pd.io.sql.read_sql(query, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг

***2. Для каждой книги посчитайте количество обзоров и среднюю оценку;***


In [15]:
query = ''' SELECT b.book_id,
            b.title,
            COUNT(DISTINCT rev.review_id) AS cnt_review,
            ROUND(AVG(r.rating), 2) AS avg_rating
            FROM books AS b
            LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
            LEFT JOIN ratings AS r ON b.book_id = r.book_id
            
            GROUP BY b.book_id, b.title
            ORDER BY cnt_review
        '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,title,cnt_review,avg_rating
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
1,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
2,221,Essential Tales and Poems,0,4.00
3,808,The Natural Way to Draw,0,3.00
4,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
...,...,...,...,...
995,733,The Giver (The Giver #1),6,3.75
996,854,The Road,6,3.77
997,695,The Curious Incident of the Dog in the Night-Time,6,4.08
998,696,The Da Vinci Code (Robert Langdon #2),6,3.83


Всего 1000 книг диапазон среднй оценки начинается от 3.66 до 4.00

***3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;***

In [10]:
query = ''' SELECT publisher,
                   publisher_id
            FROM publishers 
            WHERE publisher_id IN (SELECT publisher_id
                       FROM books
                       WHERE num_pages > 50
                       GROUP BY publisher, publisher_id
                       ORDER BY COUNT(book_id) DESC
                       LIMIT 1)  
        '''
pd.io.sql.read_sql(query, con = engine) 

,publisher,publisher_id
0,Penguin Books,212


Наибольшее число книг выпустило издательсвто Penguin Book

***4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;***

In [11]:
query = ''' WITH 
            i AS (SELECT book_id
                  FROM ratings
                  GROUP BY book_id
                  HAVING COUNT(rating_id) > 50)
            
            SELECT a.author,
                   AVG(r.rating) AS avg_rating
                   
            FROM books AS b
            JOIN i
            ON b.book_id=i.book_id
            
            JOIN author AS a
            ON b.author_id=a.author_id
            
            JOIN ratings AS r
            ON i.book_id = r.book_id
            GROUP BY a.author
            ORDER BY AVG(r.rating) DESC
             
        '''
pd.io.sql.read_sql(query, con = engine) 

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Lois Lowry,3.750000


J.K. Rowling/Mary GrandPré автор с самой высокой средней оценкой - 4.287097

***5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.***

In [12]:
query = ''' SELECT COUNT(DISTINCT rev.text)/COUNT(DISTINCT rev.username) AS avg_text
                   
            FROM ratings AS r
            INNER JOIN reviews AS rev
            ON rev.username = r.username
            WHERE r.username IN (SELECT username
                       FROM ratings
                       
                       GROUP BY username
                       HAVING COUNT(rating_id) > 50)  
        '''
pd.io.sql.read_sql(query, con = engine) 

,avg_text
0,24


Среднее количество обзоров пользователей, которые поставили больше 50 оценок -  24

***Вывод:***

В ходе анализа базы данных было выявлено:

- После 1 января 2000 года вышло 819 книг

- Для каждой книги было посчитано количество обзоров и средняя оценка: количество обзоров - минимум 0, максимум 7. Диапазон средних оценок варьируется от 3.66 до 4.0

- Наибольшее число книг выпустило издательсвто Penguin Book

- J.K. Rowling/Mary GrandPré автор с самой высокой средней оценкой - 4.287097

- Среднее количество обзоров пользователей, которые поставили больше 50 оценок - 24